In [7]:
import pandas as pd
import numpy as np
import os  
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joao_hm_silva\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [8]:
df_demanda = pd.read_csv("../../dados_preparados/dados_tratamento_inicial.csv", index_col=[0])

In [3]:
df_demanda.head()

,id_demanda,frequencia_uso,objetivo,score,situacao_atual,titulo_demanda,tipo_solicitante,cargo_solicitante,departamento_solicitante,nome_solicitante,setor_solicitante,qtd_beneficio_real,qtd_beneficio_potencial,qtd_beneficio_qualitativo,valor_total_beneficios_reais (BRL),valor_total_beneficios_potenciais (BRL),descricao_beneficios,quantidade_CC,nomes_CC
0,1,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",1.7,Duis aute irure dolor in reprehenderit in volu...,Demanda 01,Solicitante,Técnico,TI,Diego,Suporte Técnico,0,0,2,0.0,0.0,. Sit amet porttitor eget dolor morbi non arcu...,1,. Departamento de Engenharia de Sistemas
1,2,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",1.7,Duis aute irure dolor in reprehenderit in volu...,Demanda 02,Solicitante,Técnico,TI,Diego,Suporte Técnico,0,0,2,0.0,0.0,. Sit amet porttitor eget dolor morbi non arcu...,1,. Departamento de Engenharia de Sistemas
2,3,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",56.0,Duis aute irure dolor in reprehenderit in volu...,Demanda 03,Solicitante,Técnico,TI,Henrique,Administração de Redes e Sistemas Internos,0,1,1,0.0,78.0,. Sit amet porttitor eget dolor morbi non arcu...,1,. Departamento de Segurança Digital
3,4,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",56.0,Duis aute irure dolor in reprehenderit in volu...,Demanda 04,Solicitante,Técnico,TI,Henrique,Administração de Redes e Sistemas Internos,0,1,1,0.0,78.0,. Sit amet porttitor eget dolor morbi non arcu...,1,. Departamento de Segurança Digital
4,11,SEMANALMENTE,"Lorem ipsum dolor sit amet, consectetur adipis...",12.0,Duis aute irure dolor in reprehenderit in volu...,Demanda Teste Histórico 01,Solicitante,Analista de Dados,TI,Eduarda,Segurança Dital,0,0,0,0.0,0.0,NaN,1,. Departamento de Desenvolvimento de Software


In [4]:
df_demanda.columns

Index(['id_demanda', 'frequencia_uso', 'objetivo', 'score', 'situacao_atual',
       'titulo_demanda', 'tipo_solicitante', 'cargo_solicitante',
       'departamento_solicitante', 'nome_solicitante', 'setor_solicitante',
       'qtd_beneficio_real', 'qtd_beneficio_potencial',
       'qtd_beneficio_qualitativo', 'valor_total_beneficios_reais (BRL)',
       'valor_total_beneficios_potenciais (BRL)', 'descricao_beneficios',
       'quantidade_CC', 'nomes_CC'],
      dtype='object')

Colunas para o tratamento de NLP 

* frequencia_uso                 
* objetivo
* situacao_atual
* titulo_demanda
* tipo_solicitante
* cargo_solicitante
* departamento_solicitante
* nome_solicitante
* setor_solicitante
* descricao_beneficios
* nomes_CC

Colunas para a parte não NLP 

* score                 
* qtd_beneficio_real
* qtd_beneficio_potencial
* qtd_beneficio_qualitativo
* valor_total_beneficios_reais (BRL)
* valor_total_beneficios_potenciais (BRL)
* quantidade_CC

In [11]:
def adicionarNovaColuna(df_alvo, nome_coluna, df_info=df_demanda):
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('portuguese'))
    
    palavras = df_info[nome_coluna].values.astype('U').tolist()
    
    print(palavras)
    
    palavras = [palavra.lower() for palavra in palavras]
    palavras = [palavra for palavra in palavras if palavra not in stop_words]
    palavras = [stemmer.stem(palavra) for palavra in palavras]
    print(palavras)

    cvec = CountVectorizer().fit(palavras)
    df_coluna = pd.DataFrame(cvec.transform(palavras).todense(), columns=cvec.get_feature_names_out())
    
    return df_coluna

In [12]:
df_preparacao_NLP = pd.DataFrame()
nome_colunas_desejadas = [
    "frequencia_uso", "objetivo", "situacao_atual","titulo_demanda","tipo_solicitante","cargo_solicitante",
    "departamento_solicitante","nome_solicitante", "setor_solicitante","descricao_beneficios","nomes_CC"
]

for nome_coluna in df_demanda.columns:
    if nome_coluna in nome_colunas_desejadas:
        df_coluna_nova = adicionarNovaColuna(df_preparacao_NLP, nome_coluna)
        df_preparacao_NLP = pd.concat([df_preparacao_NLP, df_coluna_nova], axis=1)
        
        
df_preparacao_NLP.head()

['SEMANALMENTE', 'SEMANALMENTE', 'SEMANALMENTE', 'SEMANALMENTE', 'SEMANALMENTE', 'SEMANALMENTE', 'SEMANALMENTE', 'SEMANALMENTE', 'SEMANALMENTE', 'SEMANALMENTE', 'SEMANALMENTE', 'SEMANALMENTE', 'SEMANALMENTE', 'SEMANALMENTE', 'DIARIAMENTE', 'DIARIAMENTE', 'DIARIAMENTE', 'MENSALMENTE', 'DIARIAMENTE', 'DIARIAMENTE', 'DIARIAMENTE', 'DIARIAMENTE', 'DIARIAMENTE']
['semanalment', 'semanalment', 'semanalment', 'semanalment', 'semanalment', 'semanalment', 'semanalment', 'semanalment', 'semanalment', 'semanalment', 'semanalment', 'semanalment', 'semanalment', 'semanalment', 'diariament', 'diariament', 'diariament', 'mensalment', 'diariament', 'diariament', 'diariament', 'diariament', 'diariament']
['Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.', 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do ei

,diariament,mensalment,semanalment,11,ad,adipiscing,aliqua,aliquip,amet,atualizar,...,de,departamento,desenvolvimento,digit,engenharia,segurança,sistema,sistemas,softwar,software
0,0,0,1,0,1,1,1,1,1,0,...,2,1,0,0,1,0,1,0,0,0
1,0,0,1,0,1,1,1,1,1,0,...,2,1,0,0,1,0,1,0,0,0
2,0,0,1,0,1,1,1,1,1,0,...,1,1,0,1,0,1,0,0,0,0
3,0,0,1,0,1,1,1,1,1,0,...,1,1,0,1,0,1,0,0,0,0
4,0,0,1,0,1,1,1,1,1,0,...,2,1,1,0,0,0,0,0,1,0


In [ ]:
#tranformar o mano pra csv
# # path = "C:\\Users\\joao_hm_silva\\Documents\\GitHub\\SADEData\\dados_preparados"
# path = "C:\\Users\\joaoh\\Documents\\GitHub\\SADEData\\dados_preparados"
# os.makedirs(path, exist_ok=True)  
# df_preparacao_NLP.to_csv(path +'\dados_vetorizados_MultiProperties.csv')

In [ ]:
##pra parte de preparar a outra parte do modelo (não NLP)
df_preparacao_nao_NLP = df_demanda.drop(columns=nome_colunas_desejadas, axis=1)
df_preparacao_nao_NLP.head()

In [ ]:
df_preparacao_nao_NLP_normalizada=(df_preparacao_nao_NLP-df_preparacao_nao_NLP.min())/(df_preparacao_nao_NLP.max()-df_preparacao_nao_NLP.min())

df_preparacao_nao_NLP_normalizada.drop("id_demanda", axis=1, inplace=True)
df_preparacao_nao_NLP = pd.concat([df_preparacao_nao_NLP["id_demanda"], df_preparacao_nao_NLP_normalizada], axis=1)

df_preparacao_nao_NLP.head()

In [ ]:
#tranformar o mano a cima pra csv
# df_preparacao_nao_NLP.to_csv(path +'\dados_comuns_MultiProperties.csv')

In [ ]:
df_todos_dados = pd.concat([df_preparacao_NLP, df_preparacao_nao_NLP], axis=1)
df_todos_dados.head()

In [ ]:
# df_dados_normalizados.to_csv(path +'\dados_completos_MultiProperties.csv')